In [1]:
import time
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = path_data + '/csv'
file_name_out_r = path_csv + '/covid19-' + level + '.csv'
file_name_pop_r = path_csv + '/popolazione-regioni.csv'
file_name_icu_r = path_csv + '/terapie-intensive.csv'

In [2]:
# Define today, yesterday and day before yesterday dates
date_tday = datetime.date.today()
date_yday = Functions.get_previous_date(date_tday)
date_bf_yday = Functions.get_previous_date(date_yday)
date_tday, date_yday, date_bf_yday

(datetime.date(2020, 10, 19),
 datetime.date(2020, 10, 18),
 datetime.date(2020, 10, 17))

In [3]:
# Today url for regions file
url_tday = Functions.get_url_date(level, date_tday)
# Yesterday url for regions file
url_yday = Functions.get_url_date(level, date_yday)
# Day before yesterday url for regions file
url_bf_yday = Functions.get_url_date(level, date_bf_yday)

url_tday, url_yday, url_bf_yday

('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201019.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201018.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201017.csv')

In [4]:
# Get csv files
while True:
    try:
        # today and yesterday if today present
        df_raw_r_1 = pd.read_csv(url_tday)
        print('Presenti dati odierni')
        df_raw_r_0 = pd.read_csv(url_yday)
        date_last = date_tday
        break
    except Exception as e:
        # yesterday and day before yesterday otherwise
        df_raw_r_1 = pd.read_csv(url_yday)
        df_raw_r_0 = pd.read_csv(url_bf_yday)
        date_last = date_yday
        print('Dati di ieri')
    time.sleep(20)

Presenti dati odierni


In [5]:
# State S
df_raw_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-19T17:00:00,ITA,1,Piemonte,45.073274,7.680687,883,61,944,9663,...,774,933,30113,4203,26243,18680,44923,868770,511428,NaN
1,2020-10-19T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,26,4,30,628,...,130,135,1142,146,1746,200,1946,33852,23091,NaN
2,2020-10-19T17:00:00,ITA,3,Lombardia,45.466794,9.190347,1136,113,1249,23385,...,1340,1687,86738,17084,109408,19048,128456,2516861,1575120,NaN
3,2020-10-19T17:00:00,ITA,5,Veneto,45.434905,12.338452,396,44,440,9405,...,420,502,24253,2255,22902,13451,36353,2150361,838391,NaN
4,2020-10-19T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,62,13,75,2093,...,87,90,3973,364,6017,488,6505,478383,210178,NaN
5,2020-10-19T17:00:00,ITA,7,Liguria,44.411493,8.932699,436,34,470,3441,...,152,323,12886,1641,12884,5554,18438,378314,199539,NaN
6,2020-10-19T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,481,70,551,9119,...,507,552,27222,4518,34342,7068,41410,1383324,763923,in seguito a verifica sui dati comunicati nei ...
7,2020-10-19T17:00:00,ITA,9,Toscana,43.769231,11.255889,449,62,511,10340,...,865,986,11731,1206,18734,5054,23788,927802,624289,NaN
8,2020-10-19T17:00:00,ITA,10,Umbria,43.106758,12.388247,131,15,146,2423,...,138,167,2247,93,2099,2810,4909,254330,150026,NaN
9,2020-10-19T17:00:00,ITA,11,Marche,43.616760,13.518875,97,16,113,2005,...,73,98,6443,995,9556,0,9556,284664,166944,NaN


In [6]:
# State S - 1
df_raw_r_0.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-18T17:00:00,ITA,1,Piemonte,45.073274,7.680687,780,55,835,8998,...,953,1123,29959,4198,25943,18047,43990,859206,505300,NaN
1,2020-10-18T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,20,2,22,506,...,70,73,1137,146,1631,180,1811,33399,22840,NaN
2,2020-10-18T17:00:00,ITA,3,Lombardia,45.466794,9.190347,1065,110,1175,22119,...,1669,2975,86397,17078,108092,18677,126769,2502284,1565233,NaN
3,2020-10-18T17:00:00,ITA,5,Veneto,45.434905,12.338452,378,44,422,9003,...,685,800,24170,2256,22849,13002,35851,2145935,836440,NaN
4,2020-10-18T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,56,14,70,2011,...,142,155,3973,361,5927,488,6415,477102,209531,NaN
5,2020-10-18T17:00:00,ITA,7,Liguria,44.411493,8.932699,402,39,441,3318,...,201,370,12720,1636,12740,5375,18115,376443,198617,NaN
6,2020-10-18T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,439,67,506,8657,...,514,526,27181,4515,34047,6812,40859,1375298,760075,NaN
7,2020-10-18T17:00:00,ITA,9,Toscana,43.769231,11.255889,385,55,440,9546,...,739,906,11622,1194,17869,4933,22802,917245,617111,NaN
8,2020-10-18T17:00:00,ITA,10,Umbria,43.106758,12.388247,115,14,129,2302,...,301,327,2219,92,2062,2680,4742,253302,149410,NaN
9,2020-10-18T17:00:00,ITA,11,Marche,43.616760,13.518875,88,13,101,1944,...,183,204,6418,995,9458,0,9458,283447,166250,NaN


In [7]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-19T17:00:00,ITA,1,Piemonte,45.073274,7.680687,883,61,944,9663,...,774,933,30113,4203,26243,18680,44923,868770,511428,NaN
1,2020-10-19T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,26,4,30,628,...,130,135,1142,146,1746,200,1946,33852,23091,NaN
2,2020-10-19T17:00:00,ITA,3,Lombardia,45.466794,9.190347,1136,113,1249,23385,...,1340,1687,86738,17084,109408,19048,128456,2516861,1575120,NaN
3,2020-10-19T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,111,7,118,2435,...,145,164,8563,712,8911,2917,11828,467766,214613,NaN
4,2020-10-19T17:00:00,ITA,5,Veneto,45.434905,12.338452,396,44,440,9405,...,420,502,24253,2255,22902,13451,36353,2150361,838391,NaN
5,2020-10-19T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,62,13,75,2093,...,87,90,3973,364,6017,488,6505,478383,210178,NaN
6,2020-10-19T17:00:00,ITA,7,Liguria,44.411493,8.932699,436,34,470,3441,...,152,323,12886,1641,12884,5554,18438,378314,199539,NaN
7,2020-10-19T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,481,70,551,9119,...,507,552,27222,4518,34342,7068,41410,1383324,763923,in seguito a verifica sui dati comunicati nei ...
8,2020-10-19T17:00:00,ITA,9,Toscana,43.769231,11.255889,449,62,511,10340,...,865,986,11731,1206,18734,5054,23788,927802,624289,NaN
9,2020-10-19T17:00:00,ITA,10,Umbria,43.106758,12.388247,131,15,146,2423,...,138,167,2247,93,2099,2810,4909,254330,150026,NaN


In [8]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,11,2020-10-18T17:00:00,ITA,1,Piemonte,45.073274,7.680687,780,55,835,...,953,1123,29959,4198,25943,18047,43990,859206,505300,NaN
1,17,2020-10-18T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,20,2,22,...,70,73,1137,146,1631,180,1811,33399,22840,NaN
2,8,2020-10-18T17:00:00,ITA,3,Lombardia,45.466794,9.190347,1065,110,1175,...,1669,2975,86397,17078,108092,18677,126769,2502284,1565233,NaN
3,19,2020-10-18T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,105,7,112,...,193,219,8546,710,8802,2862,11664,464969,213540,NaN
4,18,2020-10-18T17:00:00,ITA,5,Veneto,45.434905,12.338452,378,44,422,...,685,800,24170,2256,22849,13002,35851,2145935,836440,NaN
5,5,2020-10-18T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,56,14,70,...,142,155,3973,361,5927,488,6415,477102,209531,NaN
6,7,2020-10-18T17:00:00,ITA,7,Liguria,44.411493,8.932699,402,39,441,...,201,370,12720,1636,12740,5375,18115,376443,198617,NaN
7,4,2020-10-18T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,439,67,506,...,514,526,27181,4515,34047,6812,40859,1375298,760075,NaN
8,15,2020-10-18T17:00:00,ITA,9,Toscana,43.769231,11.255889,385,55,440,...,739,906,11622,1194,17869,4933,22802,917245,617111,NaN
9,16,2020-10-18T17:00:00,ITA,10,Umbria,43.106758,12.388247,115,14,129,...,301,327,2219,92,2062,2680,4742,253302,149410,NaN


In [9]:
# Print raw columns
# print(df_r_1.columns, df_r_0.columns)
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rf_1 = df_r_1.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0 = df_r_0.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
# df_rf_1.columns, df_rf_0.columns

In [10]:
df_rf_0 = df_rf_0.sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-10-18T17:00:00,1,Piemonte,780,55,835,8998,9833,953,1123,29959,4198,25943,18047,43990,859206,505300
1,2020-10-18T17:00:00,2,Valle d'Aosta,20,2,22,506,528,70,73,1137,146,1631,180,1811,33399,22840
2,2020-10-18T17:00:00,3,Lombardia,1065,110,1175,22119,23294,1669,2975,86397,17078,108092,18677,126769,2502284,1565233
3,2020-10-18T17:00:00,4,Trentino-Alto Adige,105,7,112,2296,2408,193,219,8546,710,8802,2862,11664,464969,213540
4,2020-10-18T17:00:00,5,Veneto,378,44,422,9003,9425,685,800,24170,2256,22849,13002,35851,2145935,836440
5,2020-10-18T17:00:00,6,Friuli Venezia Giulia,56,14,70,2011,2081,142,155,3973,361,5927,488,6415,477102,209531
6,2020-10-18T17:00:00,7,Liguria,402,39,441,3318,3759,201,370,12720,1636,12740,5375,18115,376443,198617
7,2020-10-18T17:00:00,8,Emilia-Romagna,439,67,506,8657,9163,514,526,27181,4515,34047,6812,40859,1375298,760075
8,2020-10-18T17:00:00,9,Toscana,385,55,440,9546,9986,739,906,11622,1194,17869,4933,22802,917245,617111
9,2020-10-18T17:00:00,10,Umbria,115,14,129,2302,2431,301,327,2219,92,2062,2680,4742,253302,149410


In [11]:
df_r = df_rf_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(file_name_pop_r)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-10-19T17:00:00,1,Piemonte,883,61,944,9663,10607,774,933,30113,4203,26243,18680,44923,868770,511428,4341375
1,2020-10-19T17:00:00,2,Valle d'Aosta,26,4,30,628,658,130,135,1142,146,1746,200,1946,33852,23091,125501
2,2020-10-19T17:00:00,3,Lombardia,1136,113,1249,23385,24634,1340,1687,86738,17084,109408,19048,128456,2516861,1575120,10103969
3,2020-10-19T17:00:00,4,Trentino-Alto Adige,111,7,118,2435,2553,145,164,8563,712,8911,2917,11828,467766,214613,1074819
4,2020-10-19T17:00:00,5,Veneto,396,44,440,9405,9845,420,502,24253,2255,22902,13451,36353,2150361,838391,4907704
5,2020-10-19T17:00:00,6,Friuli Venezia Giulia,62,13,75,2093,2168,87,90,3973,364,6017,488,6505,478383,210178,1211357
6,2020-10-19T17:00:00,7,Liguria,436,34,470,3441,3911,152,323,12886,1641,12884,5554,18438,378314,199539,1543127
7,2020-10-19T17:00:00,8,Emilia-Romagna,481,70,551,9119,9670,507,552,27222,4518,34342,7068,41410,1383324,763923,4467118
8,2020-10-19T17:00:00,9,Toscana,449,62,511,10340,10851,865,986,11731,1206,18734,5054,23788,927802,624289,3722729
9,2020-10-19T17:00:00,10,Umbria,131,15,146,2423,2569,138,167,2247,93,2099,2810,4909,254330,150026,880285


In [12]:
# Merge with icu data
df_icu_r = pd.read_csv(file_name_icu_r)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-10-19T17:00:00,1,Piemonte,883,61,944,9663,10607,774,933,30113,4203,26243,18680,44923,868770,511428,4341375,367
1,2020-10-19T17:00:00,2,Valle d'Aosta,26,4,30,628,658,130,135,1142,146,1746,200,1946,33852,23091,125501,20
2,2020-10-19T17:00:00,3,Lombardia,1136,113,1249,23385,24634,1340,1687,86738,17084,109408,19048,128456,2516861,1575120,10103969,983
3,2020-10-19T17:00:00,4,Trentino-Alto Adige,111,7,118,2435,2553,145,164,8563,712,8911,2917,11828,467766,214613,1074819,106
4,2020-10-19T17:00:00,5,Veneto,396,44,440,9405,9845,420,502,24253,2255,22902,13451,36353,2150361,838391,4907704,825
5,2020-10-19T17:00:00,6,Friuli Venezia Giulia,62,13,75,2093,2168,87,90,3973,364,6017,488,6505,478383,210178,1211357,175
6,2020-10-19T17:00:00,7,Liguria,436,34,470,3441,3911,152,323,12886,1641,12884,5554,18438,378314,199539,1543127,209
7,2020-10-19T17:00:00,8,Emilia-Romagna,481,70,551,9119,9670,507,552,27222,4518,34342,7068,41410,1383324,763923,4467118,516
8,2020-10-19T17:00:00,9,Toscana,449,62,511,10340,10851,865,986,11731,1206,18734,5054,23788,927802,624289,3722729,415
9,2020-10-19T17:00:00,10,Umbria,131,15,146,2423,2569,138,167,2247,93,2099,2810,4909,254330,150026,880285,70


In [13]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [14]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_rf_1['ricoverati_con_sintomi'] - df_rf_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_rf_1['terapia_intensiva'] - df_rf_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_rf_1['totale_ospedalizzati'] - df_rf_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_rf_1['isolamento_domiciliare'] - df_rf_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_rf_1['nuovi_positivi'] - df_rf_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_rf_1['tamponi'] - df_rf_0['tamponi']
df_r['nuovi_casi_testati'] = df_rf_1['casi_testati'] - df_rf_0['casi_testati']
df_r['nuovi_deceduti'] = df_rf_1['deceduti'] - df_rf_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_rf_1['dimessi_guariti'] - df_rf_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-10-19T17:00:00,3,Lombardia,1136,113,1249,23385,24634,1340,1687,...,11.50,1266,-1288,14577,9887,6,341,11.57,0.01670,1.27
1,2020-10-19T17:00:00,15,Campania,884,85,969,17345,18314,1449,1593,...,19.91,1407,217,12695,9865,21,123,12.55,0.02753,0.47
2,2020-10-19T17:00:00,9,Toscana,449,62,511,10340,10851,865,986,...,14.94,794,80,10557,7178,12,109,9.34,0.02649,0.64
3,2020-10-19T17:00:00,12,Lazio,1130,111,1241,13950,15191,811,939,...,14.86,754,-259,17573,12645,9,119,5.34,0.01601,0.44
4,2020-10-19T17:00:00,1,Piemonte,883,61,944,9663,10607,774,933,...,16.62,665,-190,9564,6128,5,154,9.76,0.02149,1.03
5,2020-10-19T17:00:00,8,Emilia-Romagna,481,70,551,9119,9670,507,552,...,13.57,462,26,8026,3848,3,41,6.88,0.01236,0.93
6,2020-10-19T17:00:00,5,Veneto,396,44,440,9405,9845,420,502,...,5.33,402,-298,4426,1951,-1,83,11.34,0.01023,0.74
7,2020-10-19T17:00:00,19,Sicilia,521,72,593,6426,7019,229,362,...,13.38,199,-186,3252,2005,3,130,11.13,0.00729,0.25
8,2020-10-19T17:00:00,7,Liguria,436,34,470,3441,3911,152,323,...,16.27,123,-47,1871,922,5,166,17.26,0.02093,1.19
9,2020-10-19T17:00:00,16,Puglia,397,38,435,5028,5463,230,321,...,10.38,208,20,2446,2098,4,87,13.12,0.00801,0.29


In [15]:
# Write pre-processed data
df_r.to_csv(file_name_out_r, index=False)